In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
start_date = datetime(2001,1,1)
end_date = datetime(2022,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("strategy_speculation")

In [4]:
market.connect()
prices = market.retrieve("prices")
sp5 = market.retrieve("sp500")
market.disconnect()

In [5]:
prices = p.column_date_processing(prices)

In [6]:
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]

In [7]:
data = []
for ticker in tqdm(sp5["Symbol"]):
    ticker_data = prices[prices["ticker"]==ticker]
    ticker_data.sort_values("date",ascending=True)
    ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
    for i in range(14):
        ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
    ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=7)
    ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
    ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
    ticker_data.dropna(inplace=True)
    ticker_data["ticker"] = ticker
    data.append(ticker_data)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:55<00:00,  4.37it/s]


In [8]:
final_data = pd.concat(data)

In [9]:
factors = [str(i) for i in range(14)]
factors.append("d1")
factors.append("d2")
factors.append("d3")

In [10]:
final_data

,year,week,close,high,low,open,volume,adjclose,adjhigh,adjlow,...,8,9,10,11,12,13,d1,d2,d3,ticker
13,1994,14,99.702000,100.22800,98.750000,99.026000,3.909400e+05,11.634235,11.695614,11.523146,...,12.162794,12.295876,12.662951,12.810846,12.686328,12.359292,-0.065907,0.337379,1.867036,MMM
14,1994,15,50.252000,51.00200,50.002000,50.528000,6.364600e+05,11.727821,11.902856,11.669476,...,12.455112,12.162794,12.295876,12.662951,12.810846,12.686328,-0.057694,-0.124612,-1.369352,MMM
15,1994,16,48.000000,48.72800,47.502000,48.428000,6.473800e+05,11.202249,11.372149,11.086025,...,12.445873,12.455112,12.162794,12.295876,12.662951,12.810846,-0.075376,0.306480,-3.459481,MMM
16,1994,17,48.625000,49.19000,48.005000,48.815000,9.507000e+05,11.348111,11.479971,11.203415,...,12.115465,12.445873,12.455112,12.162794,12.295876,12.662951,-0.069841,-0.073435,-1.239608,MMM
17,1994,18,49.602000,49.80400,49.054000,49.452000,6.428600e+05,11.576124,11.623266,11.448231,...,12.200182,12.115465,12.445873,12.455112,12.162794,12.295876,-0.033589,-0.519061,6.068273,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,2022,15,120.760000,121.54000,119.535000,120.005000,2.034384e+06,120.760000,121.540000,119.535000,...,123.068044,124.220581,124.338039,121.556877,124.112082,127.347149,-0.015691,-0.557315,-5.327688,YUM
1283,2022,16,123.927500,125.43975,123.335000,124.497500,2.256762e+06,123.927500,125.439750,123.335000,...,122.685000,123.068044,124.220581,124.338039,121.556877,124.112082,0.027540,-2.755198,3.943697,YUM
1284,2022,17,118.278000,120.56000,117.588000,119.078000,1.830709e+06,118.278000,120.560000,117.588000,...,120.606000,122.685000,123.068044,124.220581,124.338039,121.556877,0.025579,-0.071201,-0.974158,YUM
1285,2022,18,116.440000,117.74000,113.340000,115.870000,2.016752e+06,116.440000,117.740000,113.340000,...,115.328000,120.606000,122.685000,123.068044,124.220581,124.338039,-0.010940,-1.427709,19.051922,YUM


In [ ]:
start_year = 2003
end_year = 2022
sim = []
speculation_db.connect()
for year in tqdm(range(start_year,end_year)):
    for ticker in tqdm(sp5["Symbol"]):
        try:
            ticker_data = final_data[final_data["ticker"]==ticker].copy()
            training_data = ticker_data[(ticker_data["year"]<year) & (ticker_data["year"]>=year-3)].reset_index(drop=True)
            training_data.dropna(inplace=True)
            prediction_data = ticker_data[ticker_data["year"]==year].dropna()
            if training_data.index.size > 100:
                refined_data = {"X":training_data[factors],"y":training_data[["adjclose"]].rename(columns={"adjclose":"y"})}
                models = m.regression(refined_data)
                for row in models.iterrows():
                    try:
                        model = row[1]["model"]
                        api = row[1]["api"]
                        score = row[1]["score"]
                        prediction_data[f"{api}_prediction"] = model.predict(prediction_data[factors])
                        prediction_data[f"{api}_score"] = score
                    except:
                        print(year,ticker,training_data.index.size)
                        continue
                included_columns = ["year","week","ticker"]
                included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
                sim.append(prediction_data[included_columns])
            else:
                continue
        except Exception as e:
            print(year,ticker,str(e))
speculation_db.disconnect()

 10%|████████████████▋                                                                                                                                                        | 50/505 [02:35<20:47,  2.74s/it]


 19%|████████████████████████████████▊                                                                                                                                        | 98/505 [04:53<09:41,  1.43s/it]

Input contains NaN, infinity or a value too large for dtype('float64').
Input contains NaN, infinity or a value too large for dtype('float64').
Input contains NaN, infinity or a value too large for dtype('float64').
[09:37:02] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/data/data.cc:981: Check failed: valid: Input data contains `inf` or `nan`



 20%|█████████████████████████████████▎                                                                                                                                      | 100/505 [04:56<10:00,  1.48s/it]

2003 CERN 156
2003 CERN 156


...

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                  | 353/505 [27:57<12:14,  4.83s/it]


 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                     | 394/505 [31:03<07:00,  3.79s/it]


 86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                       | 434/505 [34:11<06:06,  5.16s/it]


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████           | 472/505 [37:24<02:50,  5.16s/it]


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 495/505 [39:12<00:49,  4.92s/it]

In [ ]:
simulation = pd.concat(sim).groupby(["year","week","ticker"]).mean().reset_index()

In [ ]:
speculation_db.connect()
speculation_db.store("sim",simulation)
speculation_db.disconnect()